In [1]:
!python3 --version
!pip3 list

Python 3.8.10
Package                 Version
----------------------- --------------------
absl-py                 0.13.0
anyio                   3.2.1
appdirs                 1.4.3
apturl                  0.5.2
argon2-cffi             20.1.0
astunparse              1.6.3
async-generator         1.10
attrs                   21.2.0
autopep8                1.5.7
Babel                   2.9.1
backcall                0.2.0
beautifulsoup4          4.9.3
bitarray                2.3.3
bleach                  3.3.0
blinker                 1.4
Brlapi                  0.7.0
cachetools              4.2.2
certifi                 2019.11.28
cffi                    1.14.5
chardet                 3.0.4
chrome-gnome-shell      0.0.0
clang                   5.0
Click                   7.0
colorama                0.4.3
command-not-found       0.3
cryptography            2.8
cupshelpers             1.0
cycler                  0.10.0
dbus-python             1.2.16
decorator               5.0.9
defer      

# Tóm tắt quá trình
Ở project 3 bài chúng ta sẽ lần lượt làm các công việc sau:
* **Công việc 1**: Chia bô dữ liệu thành hai phần training data và test data nhằm đảm bảo tính công bằng trên mọi model.
* **Công việc 2**: Chia bài toán ban đầu thành hai bài toán con là **Emoji sentiment model** và **Comment sentiment model** và giải thích lí do.
  * **Cộng việc 2.1**: Xây dựng **Emoji sentiment model**. 
    * Xác định input và cách biểu diễn nó sau đó lựa chọn cách biểu diễn phù hợp.
    * Xác định output.
    * Tiến hành sử dụng các **Traditional Machine Learning Classifier** để đào tạo.
    * Tối ưu hóa tham số bằng phương pháp **Grid Search**.
    * Đánh giá model.
    * Lưu lại model.
  * **Công việc 2.2**: Xây dựng **Comment sentiment model**.
    * Xác định input và cách biểu diễn nó sau đó lựa chọn cách biểu diễn phù hợp.
    * Xác định output.
    * Tiến hành sử dụng các **Traditional Machine Learning Classifier** để đào tạo.
    * Tiến hành sử dũng các **Deep Learning** để đào tạo.
    * Tối ưu hóa tham số bằng phương pháp **HyperBand**.
    * Đánh giá model.
    * Lưu lại model.
* **Công việc 3**: Tổng hợp hai model **Emoji sentiment** và **Comment sentiment** để ra model cuối cùng.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import modules.model as Model
import pandas as pd

## Công việc 1:
* Phần này ta sẽ tách dữ liệu sau khi trải qua các bước tiền xử lí ở project 1 thành training data và test data. 
* Lí do ta cần thực hiện điều này là ta muốn đảm bảo công bằng cho mọi model trong quá trình đào tạo, tức chúng cùng học trên cùng một training data và được đánh giá trên cùng một test data.
* Như ở **công việc 2** đã trình bày, ta sẽ chia dữ liệu sau tiền xử lí thành hai phần:
  * **Phần dữ liệu chỉ chứa emoji**: phần dữ liệu này chỉ chứa các comment chứa emoji, các comment không chứa emoji ta sẽ loại bỏ.
  * **Phần dữ liệu chỉ chứa comment**: phần dữ liệu này chính là phần dữ liệu ban đầu nhưng khác một điều toàn bộ emoji trong comment sẽ bị xóa. 

Đọc toàn bộ dữ liệu sau khi đã tiền xử lí ở project 1.

In [4]:
data = pd.read_csv("./data/normalize_reviews.csv").fillna("")[['raw_comment', 'normalize_comment', 'label']]

data.head()

,raw_comment,normalize_comment,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,0
2,Ko có lắc tay như hình,không có lắc tay như hình,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,0


Mã hóa các dữ liệu dạng text về cùng một dạng là **NFD**.

In [5]:
data = Model.textNFxformat(data, ['raw_comment', 'normalize_comment'], 'NFD')

data.head()

,raw_comment,normalize_comment,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,0
2,Ko có lắc tay như hình,không có lắc tay như hình,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,0


Các vectorizer object của **sklearn** mặc định chúng sẽ xóa toàn bộ các **punctuation** [kí tự đặc biệt] trong input truyền vào. Như vậy các emoji của ta sẽ bị xóa toàn bộ khi transform vectorizing. Như vậy, ta sẽ không lưu chúng dưới dạng các punctuation mà dùng decode của chúng - ta sẽ tạo một feature `emoji_decode` để lưu chúng.  

In [6]:
data['emoji_decode'] = data['raw_comment'].apply(lambda s: Model.expandEmojisDecode(s))
data = data[['raw_comment', 'normalize_comment', 'emoji_decode', 'label']]

data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,cry,0


Tiến hành chọn các mẫu có feature `emoji_decode` không phải là chuổi rỗng và lưu vào biến `emoji_data`. Các mẫu trong biến này sẽ được dùng để xây dựng một **Emoji sentiment model**.

In [7]:
emoji_data = data[data['emoji_decode'] != ""].reset_index(drop=True)

emoji_data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,cry,0
1,Giao sai màu sai size có 1 dép lông size 3...,giao sai màu sai size có dép lông size à ...,cursing_face cursing_face cursing_face,0
2,Đơn hàng đã thanh toán airpay rồi mà sh...,đơn hàng thanh toán mà giao cho người l...,roll_eyes,0
3,Áo croptop freesize rộng với mình \nMìn...,áo rộng mình kg,thumbsup thumbsup thumbsup thumbsup thumbsup t...,0
4,Khá buồn . Đặt 2 cái đều không chạy ...,khá buồn đặt cái đều không chạy giơ...,female_sign shrug,0


Giờ thì ta sẽ tiến hành chia `emoji_data` thành training data và test data với size của test data là 20%, sau đó ta lưu chúng dưới dạng file **.csv**.

In [8]:
Model.dataSplitSaved(emoji_data, 0.2, "./data/emoji_data")

📢 Your dataset has saved at ./data/emoji_data.


Bây giờ ta cần chuẩn bị training data và test data cho **Comment sentiment model**, ta cũng sẽ chia tập dữ liệu sau tiền xử lí thành hai phần training data và test data với test data chiếm 20% dữ liệu sau tiền xử lí. Cuối cùng ta cũng sẽ lưu hai tập training data và test data này dưới dạng file **.csv**.

In [9]:
data.head()

,raw_comment,normalize_comment,emoji_decode,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,cry,0


In [10]:
Model.dataSplitSaved(data, 0.2, "./data/data")

📢 Your dataset has saved at ./data/data.


## Công việc 2:
* Ở phần này, nhóm sẽ trình bày về các vấn đề sau:
  * **Vấn đề 1**: Lựa chọn thuật toán tương ứng lần lượt cho hai model và lí giải.
  * **Vấn đề 2**: Lựa chọn kĩ thuật đánh giá. 

### Vấn đề 1:
* Bài toán của chúng ta là NLP - như vậy thì dữ liệu sẽ khiến cho chúng ta khó hiểu hơn về dữ liệu. Nhưng theo những gì nhóm đã được học ở những môn trước thì nhóm có hi vọng cao vào hai thuật toán là **Logistic Regression** và **Support Vector Machine**. Tuy nhiên nhóm vẫn sẽ áp dụng các model classification khác như Naive Bayes, Random Forest,... vì khả năng cao chúng có thể đại diện tốt cho dataset của chúng ta.

#### Logistic Regession:
* Ở phần đào tạo mô hình sau này, nhóm sẽ ưu tiên sử dụng thuật toán này trên các `solver` khác nhau như `newton-cg`, `lbfgs`, `liblinear`. Đây là một sự ưu tiên cho thuật toán này vì nhóm nghĩ nó hiệu quả vì:
  * Bài toán của chúng ta là **binary classification** và **Logistic Regression** thường được coi là thuật toán cơ bản nhất cho các bài toán dạng này.
  * Thuật toán **Logistic Regression** có thời gian thực thi nhanh và cách cài đặt đơn giản, các **hyper-parameter** không nhiều nên dễ dàng thực hiện kĩ thuật **Tunning Hyper-Parameters**.
  * Đối với **Emoji sentiment model**, thực chất số lượng emoji mà người dùng hay dùng không nhiều, số emoji trong một comment cũng không nhiều $\Rightarrow$ Khiến cho dữ liệu đào tạo đơn giản và dễ hiệu nên **Logistic Regression** rất phù hợp với các dataset đơn giản như vậy đồng thời sẽ cho ra độ chính xác cao.
  * Với **Comment sentiment model** - dữ liệu phức tạp hơn nhưng chúng ta cũng nên kì vọng là thuật toán này sẽ hoạt động tốt.
  
#### Support Vector Machine
* Do nhóm nghĩ đây là một thuật toán hiệu quả, nên nhóm cũng sẽ có chút ưu tiên cho thuật toán bày bằng cách triển khai nó trên nhiều `kernel` khác nhau như `linear`, `poly`, `rbf`, `sigmoid`. Lí do nhóm ưu tiên thuật toán này là vì:
  * Với các dữ liệu mà ta khó có cái nhìn tổng quan hoặc ý tưởng thì SVM là một mô hình khá tốt để ta tiến hành đào tạo vì nó linh hoạt - có thể dùng cho hai bài toán là **regression** và **classification** thậm chí là cho cả các bài toán **clustering**.
  * Nó hoạt động tốt trên dữ liệu phức tạp mà với dữ liệu text thì text hay được biểu diễn dưới dạng vector.
  * Với các bài toán phân lớp, nó sử dụng các `kernel` để đưa input đầu vào vào một không gian có nhiều chiều hơn ngoài ra còn cố gắng tối đa hóa khoảng cách giữa **sepertating hyperplan** với các **super vectors**.<br>
    ![](./images/10.png)
  * Hoạt động hiệu quả trên bài toán phân loại văn bản, dữ liệu phi cấu trúc và nhiều chiều.
  * Ngoài ra, sức mạnh của thuật toán này chính là dựa trên các `kernel` mà ta lựa chọn, tuy nhiên để chọn ra `kernel` tốt không dễ dàng nên ta thường vét cạn, nhưng nếu dữ liệu quá lớn thì không nên vì thời gian đào tạo của thuật toán này lâu, có thể nói ngang ngữa với các **Deep Neural Network**.

#### Deep Neural Netword
* Các thuật toán thuộc nhóm DNN sẽ được trình bày sau. Nhóm sẽ tập trung trình bày vào LSTM vì đây là model hoạt động tốt nhất trên dataset của nhóm.

### Vấn đề 2:
![](./images/11.png)
* Chúng ta sẽ sử dụng hai độ đo phổ biến nhất dành cho các classification model là:
  * **Accuracy**: dùng để đánh giá độ chính xác của model trên **TN** và **TP**.
  * **ROC-AUC**: accuracy sẽ không chính xác nếu như số lượng mẫu giữa các class bị mất cân năng nên ROC-AUC sẽ giúp ta kiểm tra việc xem có một class nào nổi trội hơn so với class còn lại không.

Bây giờ ta sẽ tiến hành build một **Emoji sentiment model**